In [1]:
# Imports
from time import time
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate

In [2]:
atoms = ['li','be','n','ne','na','mg','p','ar','k','ca','zn','as','kr','rb','sr','cd','sb','xe']

In [3]:
tdat = 0
ne = []
ri = []
rho = []
KED = []
pop = []
gradrho = []
laprho = []
gradrhogradgradrhogradrho = []
gradrhogradlaprho = []
lap2rho = []
gradgradrho2 = []
modgradrhogradgradrho2 = []
modgradlaprho2 = []
gradlaprhogradgradrhogradrho = []
for atom in atoms: 
    file = str('/home/lrincon/dl_ked/pinn/data_pinn/'+atom+'.dat') 
    f = open(file,'r') 
    linea = f.readline() 
    fields = linea.split() 
    ndat = int(fields[0]) 
    nel = int(fields[1]) 
    tdat = tdat + ndat
    print(ndat,nel) 
    for i in range(ndat): 
        linea = f.readline() 
        fields = linea.split()
        ne.append(nel)
        ri.append(float(fields[0])) 
        rho.append(float(fields[1]))
        KED.append(float(fields[2]))  
        pop.append(float(fields[3]))
        gradrho.append(float(fields[4])) 
        laprho.append(float(fields[5]))
        gradrhogradgradrhogradrho.append(float(fields[6]))
        gradrhogradlaprho.append(float(fields[7]))
        lap2rho.append(float(fields[8]))
        gradgradrho2.append(float(fields[9]))
        modgradrhogradgradrho2.append(float(fields[10]))
        modgradlaprho2.append(float(fields[11]))
        gradlaprhogradgradrhogradrho.append(float(fields[12]))
    f.close()
print(tdat)

108567 3
86028 4
59757 7
46683 10
114353 11
98148 12
79005 15
63195 18
130898 19
115705 20
86324 30
83240 33
70946 36
136914 37
122591 38
93142 48
90966 51
79378 54
1665840


In [4]:
data = np.ndarray(shape=(tdat,14))
zeta = 2.0*np.power(3.0*np.power(np.pi,2),1.0/3.0)
for i in range(tdat):
    data[i][0] = KED[i]
    data[i][1] = pop[i]
    data[i][2] = ri[i]
    data[i][3] = ne[i]
    data[i][4] = rho[i]
    z = gradrho[i]/(zeta*np.power(rho[i],4.0/3.0))
    data[i][5] = np.power(z,2)
    z = laprho[i]/(np.power(zeta,2)*np.power(rho[i],5.0/3.0))
    data[i][6] = z
    z = gradrhogradgradrhogradrho[i]/(np.power(zeta,4)*np.power(rho[i],13.0/3.0))
    data[i][7] = z
    z = gradrhogradlaprho[i]/(np.power(zeta,4)*np.power(rho[i],10.0/3.0))
    data[i][8] = z
    z = lap2rho[i]/(np.power(zeta,4)*np.power(rho[i],7.0/3.0))
    data[i][9] = z
    z = gradgradrho2[i]/(np.power(zeta,4)*np.power(rho[i],10.0/3.0))
    data[i][10] = z
    z = modgradrhogradgradrho2[i]/(np.power(zeta,6)*np.power(rho[i],6))
    data[i][11] = z
    z = modgradlaprho2[i]/(np.power(zeta,6)*np.power(rho[i],4))
    data[i][12] = z
    z = gradlaprhogradgradrhogradrho[i]/(np.power(zeta,6)*np.power(rho[i],5))
    data[i][13] = z
np.random.shuffle(data)

In [5]:
# normalization function: f_norm
def f_norm(x):
    return x/(1.0+x)
# inverse
def fi_norm(x):
    return x/(1.0-x)
def df_norm(x):
    return 1.0/np.power(1.0+x,2)
def dfi_norm(x):
    return 1.0/np.power(1.0-x,2)

In [6]:
# Normalization
fraction = 0.010
dim = np.int(fraction*tdat)
print(dim)
# Target data: 0: ked, 1: potential
ked_target = np.ndarray(shape=(dim,2))
# Descriptors: 0: number of electron, 1: reduced gradient, 2: reduced laplacian
ked_input = np.ndarray(shape=(dim,3))
for i in range(dim):
    KEDi = data[i][0]
    popi = data[i][1]
    nei = data[i][3]
    rhoi = data[i][4]
    s2 = data[i][5]
    p = data[i][6]
    CTF = (3.0/10.0)*np.power(3.0*np.power(np.pi,2),2.0/3.0)
    TF = CTF*np.power(rhoi,5.0/3.0)
    vW = (5.0/3.0)*TF*s2
    delta = (KEDi-TF-(1.0/9.0)*vW)/TF
    ked_target[i][0] = delta
    ked_target[i][1] = (popi/(CTF*np.power(rhoi,2.0/3.0)))-(5.0/3.0)-(5.0/27.0)*s2+(10.0/27.0)*p
    ked_input[i][0] = nei/100.0
    ked_input[i][1] = s2
    ked_input[i][2] = p
ked_target = tf.convert_to_tensor(ked_target,dtype='float32')
ked_input = tf.convert_to_tensor(ked_input,dtype='float32')

16658


In [7]:
a1 = np.ndarray(dim)
a2 = np.ndarray(dim)
a3 = np.ndarray(dim)
a4 = np.ndarray(dim)
a5 = np.ndarray(dim)
a6 = np.ndarray(dim)
a7 = np.ndarray(dim)
a8 = np.ndarray(dim)
a9 = np.ndarray(dim)
for i in range(dim):
    s2 = data[i][5]
    p  = data[i][6]
    q0 = data[i][7]
    q1 = data[i][8]
    q2 = data[i][9]
    q3 = data[i][10]
    h0 = data[i][11]
    h1 = data[i][12]
    h2 = data[i][13]
    a1[i] = 5.0/3.0
    a2[i] = -((2.0/3.0)*s2+2.0*p)
    a3[i] = -(5.0/3.0)*p
    a4[i] = (16.0/3.0)*np.power(s2,2)-4.0*q0
    a5[i] = (88.0/9.0)*np.power(s2,2)+(2.0/3.0)*s2*p-(32.0/3.0)*q0+2.0*q3
    a6[i] = (40.0/9.0)*s2*p-(5.0/3.0)*np.power(p,2)-(10.0/3.0)*q1+q2
    a7[i] = (64.0/9.0)*np.power(s2,3)-(32.0/3.0)*s2*q0+4.0*h0
    a8[i] = (80.0/9.0)*np.power(s2,2)*p-(16.0/3.0)*s2*q1-(20.0/3.0)*p*q0+4.0*h2
    a9[i] = (25.0/9.0)*s2*np.power(p,2)-(10.0/3.0)*p*q1+h1
a1 = tf.convert_to_tensor(a1,dtype='float32')
a2 = tf.convert_to_tensor(a2,dtype='float32')
a3 = tf.convert_to_tensor(a3,dtype='float32')
a4 = tf.convert_to_tensor(a4,dtype='float32')
a5 = tf.convert_to_tensor(a5,dtype='float32')
a6 = tf.convert_to_tensor(a6,dtype='float32')
a7 = tf.convert_to_tensor(a7,dtype='float32')
a8 = tf.convert_to_tensor(a8,dtype='float32')
a9 = tf.convert_to_tensor(a9,dtype='float32')

In [8]:
# Define residual of the potential
def pop_res(u, du_ds, du_dp, d2u_ds2, d2u_dsp, d2u_dp2, d3u_ds2p, d3u_dsp2, d3u_dp3, a1, a2, a3, a4, a5, a6, a7, a8, a9):
    product1 = tf.math.multiply(a1,u)
    product2 = tf.math.multiply(a2,du_ds)
    product3 = tf.math.multiply(a3,du_dp)
    product4 = tf.math.multiply(a4,d2u_ds2)
    product5 = tf.math.multiply(a5,d2u_dsp)
    product6 = tf.math.multiply(a6,d2u_dp2)
    product7 = tf.math.multiply(a7,d3u_ds2p)
    product8 = tf.math.multiply(a8,d3u_dsp2)
    product9 = tf.math.multiply(a9,d3u_dp3)
    return product1 + product2 + product3 + product4 + product5 + product6 + product7 + product8 + product9

In [9]:
def KED_model(input_shape = (3)):
    inputs = Input(shape=input_shape)
    x = Dense(50,activation='relu')(inputs)
    x = Dense(50,activation='relu')(x)
    x = Dense(50,activation='relu')(x)
    x = Dense(50,activation='relu')(x)
    x = Concatenate()([inputs, x])
    x = Dense(25,activation='relu')(x)
    x = Dense(1,activation='relu')(x)
    KED = tf.keras.Model(inputs=inputs, outputs= x)    
    return KED

In [10]:
model = KED_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 50)           200         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           2550        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 50)           2550        dense_1[0][0]                    
______________________________________________________________________________________________

In [11]:
# determine gradient
def get_r(model,ked_input,a1,a2,a3,a4,a5,a6,a7,a8,a9):
    
    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split x_train to compute partial derivatives
        ne_norm,s2_norm, p_norm = ked_input[:,0:1], ked_input[:,1:2], ked_input[:,2:3]

        # Variables s_norm and p_norm are watched during tape
        # to compute derivatives du_ds and du_dp
        tape.watch(ne_norm)
        tape.watch(s2_norm)
        tape.watch(p_norm)

        # Determine residual 
        u = model(tf.stack([ne_norm[:,0],s2_norm[:,0],p_norm[:,0]],axis=1))

        # Compute gradient du_dx and du_dp within the GradientTape
        # since we need second derivatives
        du_ds = tape.gradient(u, s2_norm)
        du_dp = tape.gradient(u, p_norm)    
        d2u_ds2 = tape.gradient(du_ds, s2_norm)
        d2u_dsp = tape.gradient(du_ds, p_norm)
        d2u_dp2 = tape.gradient(du_dp, p_norm)
    # Compute the second derivative
    d3u_ds2p = tape.gradient(d2u_ds2, p_norm)
    d3u_dsp2 = tape.gradient(d2u_dsp, p_norm)
    d3u_dp3  = tape.gradient(d2u_dp2, p_norm)

    del tape
    
    # Compute residual
    return pop_res(u, du_ds, du_dp, d2u_ds2, d2u_dsp, d2u_dp2, d3u_ds2p, d3u_dsp2, d3u_dp3, a1, a2, a3, a4, a5, a6, a7, a8, a9)

In [12]:
# compute the loss
def compute_loss(model,ked_input,ked_target,a1,a2,a3,a4,a5,a6,a7,a8,a9):
    
    # Compute the residual
    r = get_r(model,ked_input,a1,a2,a3,a4,a5,a6,a7,a8,a9)
    phi_r = tf.reduce_mean(tf.square(r-ked_target[:,1]))
    
    # Initialize loss with the residual
    loss = phi_r
    
    # Add the error in KED to the loss
    for i in range(len(ked_input)):
        x_input = tf.expand_dims(ked_input[i],axis=0)
        u_pred = model(x_input)
        loss += tf.reduce_mean(tf.square(u_pred-ked_target[i,0]))
    
    return loss

In [13]:
# Computer Neural Network trainable variables gradient
def get_grad(model,ked_input,ked_target,a1,a2,a3,a4,a5,a6,a7,a8,a9):
    
    with tf.GradientTape(persistent=True) as tape:
        # This tape is for derivatives with
        # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model,ked_input,ked_target,a1,a2,a3,a4,a5,a6,a7,a8,a9)

    g = tape.gradient(loss,model.trainable_variables)
    del tape

    return loss, g

In [14]:
# Set up optimizer
# Initialize model
model = KED_model()

# We choose a piecewise decay of the learning rate, i.e., the
# step size in the gradient descent type algorithm
# the first 1000 steps use a learning rate of 0.01
# from 1000 - 3000: learning rate = 0.001
# from 3000 onwards: learning rate = 0.0005

lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])

# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
# train the model
# Define one training step as a TensorFlow function to increase speed of training
@tf.function
def train_step():
    # Compute current loss and gradient w.r.t. parameters
    loss, grad_theta = get_grad(model,ked_input,ked_target,a1,a2,a3,a4,a5,a6,a7,a8,a9)
    
    # Perform gradient descent step
    optim.apply_gradients(zip(grad_theta, model.trainable_variables))
    
    return loss

# Number of training epochs
N = 5000
hist = []

for i in range(N+1):
    
    t0 = time()
    
    loss = train_step()
    
    # Append current loss to hist
    hist.append(loss.numpy())
    
    # print the loss
    print('Iteration {:06d} : (loss = {:10.8e} , time = {} )'.format(i,loss,time()-t0))
